# ***Case 1***

In [0]:
#@title Import packages
import pandas as pd
import numpy as np

from tabulate import tabulate


from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


## ***Data Input***

In [67]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

link = 'https://drive.google.com/open?id=16cn80fQQ9cEXAFYW_b9rm6rRMfKDDWjc'
fluff, id = link.split('=')
file = drive.CreateFile({'id':id}) 
file.GetContentFile('loan2.csv') 
DSLoanTrain = pd.read_csv('loan2.csv')
DSLoanTrain.head()

,id,loan_amnt,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,policy_code,loan_status
0,1077501,5000,5000,36 months,10.65,162.87,B,B2,10+ years,3,24000.0,3,27.65,0,31048,1,NaN,NaN,3,0,13648,83.7,9,0.0,0.0,1,2
1,1077430,2500,2500,60 months,15.27,59.83,C,C4,< 1 year,3,30000.0,2,1.00,0,36251,5,NaN,NaN,3,0,1687,9.4,4,0.0,0.0,1,1
2,1077175,2400,2400,36 months,15.96,84.33,C,C5,10+ years,3,12252.0,1,8.72,0,37196,2,NaN,NaN,2,0,2956,98.5,10,0.0,0.0,1,2
3,1076863,10000,10000,36 months,13.49,339.31,C,C1,10+ years,3,49200.0,2,20.00,0,35096,1,35.0,NaN,10,0,5598,21.0,37,0.0,0.0,1,2
4,1075269,5000,5000,36 months,7.90,156.46,A,A4,3 years,3,36000.0,2,11.20,0,38292,3,NaN,NaN,9,0,7963,28.3,12,0.0,0.0,1,2


# ***Clean Data***

In [0]:
CleanData = DSLoanTrain.select_dtypes(include=[np.number]).interpolate().dropna()

yPredict = CleanData.loan_status
XClean = CleanData.drop(["loan_status"], axis=1)

# ***Setup Traning***

In [0]:
X_train, X_test, y_train, y_test = train_test_split(XClean, yPredict, random_state=42, test_size=.33)
clf = RandomForestRegressor(n_jobs=2, n_estimators=1000)
model = clf.fit(X_train, y_train)


# ***Feature Importance***

In [8]:
headers = ["name", "score"]
values = sorted(zip(X_train.columns, model.feature_importances_), key=lambda x: x[1] * -1)
print(tabulate(values, headers, tablefmt="plain"))

name                          score
int_rate                0.1132
dti                     0.0799081
mths_since_last_record  0.0785703
id                      0.0784483
annual_inc              0.0778227
earliest_cr_line        0.0774411
mths_since_last_delinq  0.0753999
revol_util              0.0751632
revol_bal               0.0750679
installment             0.056094
total_acc               0.0536633
open_acc                0.0408344
loan_amnt               0.0316979
funded_amnt             0.0311828
inq_last_6mths          0.01973
verification_status     0.0122382
home_ownership          0.0106782
pub_rec                 0.00654515
delinq_2yrs             0.00620503
out_prncp_inv           6.31813e-05
out_prncp               4.65174e-05
policy_code             0


# ***Case 2***

In [70]:
link = 'https://drive.google.com/open?id=1VOAhrB5eIhFc8C5qeFx5QYa2OLD7s1eV'
fluff, id = link.split('=')
file = drive.CreateFile({'id':id}) 
file.GetContentFile('loan-clean-version.csv') 
DSLoanTrain2  = pd.read_csv('loan-clean-version.csv')
DSLoanTrain2 .head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,purpose,addr_state,dti,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,loan_status
0,1077501,5000,5000,4975.0,36 months,10.65,162.87,B,10+ years,RENT,24000.0,Verified,credit_card,AZ,27.65,31048,1,3,0,13648,83.7,9,0,0,5861.071414,5831.78,5000.00,861.07,Fully Paid
1,1077430,2500,2500,2500.0,60 months,15.27,59.83,C,Less than 1 year,RENT,30000.0,Source Verified,car,GA,1.00,36251,5,3,0,1687,9.4,4,0,0,1008.710000,1008.71,456.46,435.17,Charged Off
2,1077175,2400,2400,2400.0,36 months,15.96,84.33,C,10+ years,RENT,12252.0,Not Verified,small_business,IL,8.72,43040,2,2,0,2956,98.5,10,0,0,3003.653644,3003.65,2400.00,603.65,Fully Paid
3,1076863,10000,10000,10000.0,36 months,13.49,339.31,C,10+ years,RENT,49200.0,Source Verified,other,CA,20.00,35096,1,10,0,5598,21.0,37,0,0,12226.302210,12226.30,10000.00,2209.33,Fully Paid
4,1075269,5000,5000,5000.0,36 months,7.90,156.46,A,3 years,RENT,36000.0,Source Verified,wedding,AZ,11.20,43043,3,9,0,7963,28.3,12,0,0,5631.377753,5631.38,5000.00,631.38,Fully Paid


# ***Data Cleaning / Integration***

In [0]:

class_mapping = {label:idx for idx, label in enumerate(np.unique(DSLoanTrain2['term']))}
DSLoanTrain2['term']=DSLoanTrain2['term'].map(class_mapping)
class_mapping = {label:idx for idx, label in enumerate(np.unique(DSLoanTrain2['grade']))}
DSLoanTrain2['grade']=DSLoanTrain2['grade'].map(class_mapping)
class_mapping = {label:idx for idx, label in enumerate(np.unique(DSLoanTrain2['home_ownership']))}
DSLoanTrain2['home_ownership']=DSLoanTrain2['home_ownership'].map(class_mapping)

class_mapping = {label:idx for idx, label in enumerate(np.unique(DSLoanTrain2['verification_status']))}
DSLoanTrain2['verification_status']=DSLoanTrain2['verification_status'].map(class_mapping)

class_mapping = {label:idx for idx, label in enumerate(np.unique(DSLoanTrain2['purpose']))}
DSLoanTrain2['purpose']=DSLoanTrain2['purpose'].map(class_mapping)

class_mapping = {label:idx for idx, label in enumerate(np.unique(DSLoanTrain2['addr_state']))}
DSLoanTrain2['addr_state']=DSLoanTrain2['addr_state'].map(class_mapping)

class_mapping = {label:idx for idx, label in enumerate(np.unique(DSLoanTrain2['loan_status']))}
DSLoanTrain2['loan_status']=DSLoanTrain2['loan_status'].map(class_mapping)

In [56]:
from sklearn.model_selection import train_test_split

DSLoanTrain2.head()



,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,purpose,addr_state,dti,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,loan_status
0,1077501,5000,5000,4975.0,0,10.65,162.87,1,10+ years,2,24000.0,2,1,3,27.65,31048,1,3,0,13648,83.7,9,0,0,5861.071414,5831.78,5000.00,861.07,1
1,1077430,2500,2500,2500.0,1,15.27,59.83,2,Less than 1 year,2,30000.0,1,0,10,1.00,36251,5,3,0,1687,9.4,4,0,0,1008.710000,1008.71,456.46,435.17,0
2,1077175,2400,2400,2400.0,0,15.96,84.33,2,10+ years,2,12252.0,0,10,12,8.72,43040,2,2,0,2956,98.5,10,0,0,3003.653644,3003.65,2400.00,603.65,1
3,1076863,10000,10000,10000.0,0,13.49,339.31,2,10+ years,2,49200.0,1,8,4,20.00,35096,1,10,0,5598,21.0,37,0,0,12226.302210,12226.30,10000.00,2209.33,1
4,1075269,5000,5000,5000.0,0,7.90,156.46,0,3 years,2,36000.0,1,12,3,11.20,43043,3,9,0,7963,28.3,12,0,0,5631.377753,5631.38,5000.00,631.38,1


In [0]:
DSLoanTrain2 = DSLoanTrain2.select_dtypes(include=[np.number]).interpolate().dropna()
DSLoanTrain2 = DSLoanTrain2.drop(["total_pymnt"], axis=1)
DSLoanTrain2 = DSLoanTrain2.drop(["total_pymnt_inv"], axis=1)
DSLoanTrain2 = DSLoanTrain2.drop(["total_rec_int"], axis=1)



# ***Split files***

In [0]:
yPredict = DSLoanTrain2.loan_status
XClean = DSLoanTrain2.drop(["loan_status"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(XClean, yPredict, random_state=42, test_size=.30)


In [59]:
yPredict.head()

0    1
1    0
2    1
3    1
4    1
Name: loan_status, dtype: int64

# ***Transform File***

In [0]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# ***Nerual Network***

In [61]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(30,30,30))
mlp.fit(X_train,y_train)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(30, 30, 30), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

# ***Prediction***

In [62]:
predictions = mlp.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,predictions))


[[ 444   12]
 [   1 2245]]


# ***Evaluation***

In [63]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       1.00      0.97      0.99       456
           1       0.99      1.00      1.00      2246

    accuracy                           1.00      2702
   macro avg       1.00      0.99      0.99      2702
weighted avg       1.00      1.00      1.00      2702

